# Line Setup Changeover Detection Challenge

This exercise is designed to give you a chance to see the kind of work that you would be doing as an Applications Engineer at Arch.  You've been given a synthesized dataset that corresponds to data over several days from electronics assembly machines called "Pick and Place" machines.  The synthetic data you've been given is designed to mimic the data that is produced by ASM P&P Machine.

For context, in a factory of SMT lines, there are an arbitrary number of Pick and Place machines that are in a sequential line. Each machine is assigned a Station ID. There are some times where a customer may move a machine to a diferent location. 

As part of the ASM protocol, the machines are all produce heartbeats that determines what machines are online.

If you want you can read more about ASM PNP machines, [on their website](https://www.asm-smt.com/en/).  **Note:** There is no need to be familiar with this specific brand of P&P machine, or even P&P machines in general.  The link was just provided in case you're interested in learning more.

## Goal

The goal of this challenge is to query the provided SQL database, which is running in docker using Apache Drill to determine:

- The current machines assigned for each line
- Which machine is no longer assigned
- Which machine replace the unused machine.


### Expected Outcome and How To Return Your Solution

The expected outcome is that this Jupyter notebook is filled in with **working python code** that:

- gets the required data from the parquet file into a Pandas DataFrame
- operates on the Pandas DataFrame to extract the needed information
- prints the solution.

Once you've completed the notebook, render it as HTML so it can be viewed easily and return **both the html rendered notebook and the .ipynb notebook file itself** in a zip folder by email.  The notebook should have both the inputs and output cells visible so its clear what commands you ran and what the result was.

In [1]:
import pandas as pd

## Configurable Settings

Enter the path to where you saved the parquet data file here.  The following cells validate that the data was loaded correctly.  You should see 129948 rows.


In [2]:
DATAFILE_PATH = '../datasets/factory_asm_1week_acme_2022_11_07.snappy.parquet'
raw_data = pd.read_parquet(DATAFILE_PATH)

In [3]:
raw_data.iloc[0].T

monitoringservicesendtime    2022-11-06T16:32:35.3242268Z
linefullpath                                System\LINE_5
linename                                           LINE_5
machineid                                         16067.0
stationfullpath                    System\LINE5\O5M1_X4is
stationid                                           16067
stationname                                     O5M1_X4is
stationorder                                            1
typename                                    SIPLACE X4i S
Name: 0, dtype: object

In [4]:
len(raw_data)

129948

# Your Solution

Please **fill in your solution** below in the next cell, leaving any cells that are relevant to understanding how you calculated your result.  The final cell should be a printed Pandas DataFrame as shown in the example.

## Understanding the Data

Everything you need to know in order to solve this challenge can be found in the data itself or the hints and links provided in this description.



The dataset you have been given is structured as follows:

- each row is a heartbeat eventlog produced by each machine every 5 minutes.  
- There are multiple machines per line.
- each row is a complex nested object containing rich data on how this machine is configured
- some of the information is not needed.  However, there are a few key data fields inside the array that are relevant:
   - `monitoringservicesendtime`: The ISO8601 timestamp as a string of when this machine finished producing this board
   - `linefullpath`: a stable identifier for the factory line that contains the machine that generated this data
   - `linename`: abbreviated name for the line
   - `stationfullpath`: a stable identifier (globally unique) for the specific machine that generated this data
   - `stationname`: abbreviated name for the machine
   - `stationid`: serial number assosciated with the machine
   - `stationorder`: which order the machine lies in the line.

In [5]:
## Fill in your implementation here.

In [6]:
raw_data

,monitoringservicesendtime,linefullpath,linename,machineid,stationfullpath,stationid,stationname,stationorder,typename
0,2022-11-06T16:32:35.3242268Z,System\LINE_5,LINE_5,16067.0,System\LINE5\O5M1_X4is,16067,O5M1_X4is,1,SIPLACE X4i S
1,2022-11-06T16:32:35.3242268Z,System\LINE_5,LINE_5,10000068.0,System\LINE5\O5M2_D4,10000068,O5M2_D4,2,SIPLACE D4
2,2022-11-06T16:32:35.3242268Z,System\LINE_5,LINE_5,13942.0,System\LINE5\O5M3_X4is,13942,O5M3_X4is,3,SIPLACE X4i S
3,2022-11-06T16:32:35.3242268Z,System\LINE_5,LINE_5,26.0,System\LINE5\O5M4_D1,26,O5M4_D1,4,SIPLACE D1
4,2022-11-06T16:32:35.3242268Z,System\LINE_8,LINE_8,4885.0,System\LINE8\O8M1_X4,4885,O8M1_X4,1,SIPLACE X4
...,...,...,...,...,...,...,...,...,...
129943,2022-11-12T06:26:01.6368476Z,System\LINE_2,LINE_2,202.0,System\LINE2\O2M1_D4,202,O2M1_D4,1,SIPLACE D4
129944,2022-11-12T06:26:01.6368476Z,System\LINE_2,LINE_2,2560.0,System\LINE2\O2M2_D1,2560,O2M2_D1,2,SIPLACE D1
129945,2022-11-12T06:26:01.6368476Z,System\LINE_2,LINE_2,10002712.0,System\LINE2\O2M3_D1,10002712,O2M3_D1,3,SIPLACE D1
129946,2022-11-12T06:26:01.6368476Z,System\LINE_4,LINE_4,5899.0,System\LINE4\O4M1_X4i,5899,O4M1_X4i,1,SIPLACE X4I


In [7]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129948 entries, 0 to 129947
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   monitoringservicesendtime  129948 non-null  object 
 1   linefullpath               129948 non-null  object 
 2   linename                   129948 non-null  object 
 3   machineid                  128364 non-null  float64
 4   stationfullpath            129948 non-null  object 
 5   stationid                  129948 non-null  int64  
 6   stationname                129948 non-null  object 
 7   stationorder               129948 non-null  int64  
 8   typename                   129948 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 8.9+ MB


In [8]:
raw_data.isnull().sum()

monitoringservicesendtime       0
linefullpath                    0
linename                        0
machineid                    1584
stationfullpath                 0
stationid                       0
stationname                     0
stationorder                    0
typename                        0
dtype: int64

# The current machines assigned for each line

In [13]:
machine_data = pd.DataFrame(raw_data.groupby('linename').stationname.value_counts())
machine_data
#pd.pivot_table(raw_data, index=['linename', 'stationname'])

stationname
linename stationname             
LINE_1   O1M1_X4is           6009
         O1M2_X3s            6009
LINE_2   O2M1_D4             6009
         O2M2_D1             6009
         O2M3_D1             6009
LINE_4   O4M1_X4i            6009
         O4M2_X4             6009
LINE_5   O5M1_X4is           6009
         O5M2_D4             6009
         O5M3_X4is           6009
         O5M4_D1             6009
LINE_6   O6M1_X4is           6009
         O6M2_D4             6009
         O6M3_X4is           6009
         O6M4_D1             6009
LINE_7   O7M1_X4is           1803
         O7M2_SX             1803
LINE_8   O8M2_SX             6009
         O8M3_D1             6009
         O8M1_TX             4359
         O8M1_X4             1803
LINE_9   O9M1_TX             6009
         O9M2_TX             6009
         O9M3_SX             6009

# Which machine is no longer assigned

In [13]:
new_data = raw_data[raw_data['machineid'].isnull()]
new_data

,monitoringservicesendtime,linefullpath,linename,machineid,stationfullpath,stationid,stationname,stationorder,typename
3280,2022-11-10T00:31:22.6198279Z,System\LINE_8,LINE_8,NaN,System\LINE8\O8M1_TX,137156501,O8M1_TX,1,SIPLACE TX
3301,2022-11-10T00:31:22.6198279Z,System\LINE_8,LINE_8,NaN,System\LINE8\O8M1_TX,137156501,O8M1_TX,1,SIPLACE TX
3322,2022-11-10T00:31:22.6198279Z,System\LINE_8,LINE_8,NaN,System\LINE8\O8M1_TX,137156501,O8M1_TX,1,SIPLACE TX
3343,2022-11-10T00:36:22.9446352Z,System\LINE_8,LINE_8,NaN,System\LINE8\O8M1_TX,137156501,O8M1_TX,1,SIPLACE TX
3364,2022-11-10T00:36:22.9446352Z,System\LINE_8,LINE_8,NaN,System\LINE8\O8M1_TX,137156501,O8M1_TX,1,SIPLACE TX
...,...,...,...,...,...,...,...,...,...
129055,2022-11-09T12:20:05.4207979Z,System\LINE_8,LINE_8,NaN,System\LINE8\O8M1_TX,137156501,O8M1_TX,1,SIPLACE TX
129076,2022-11-09T12:20:05.4207979Z,System\LINE_8,LINE_8,NaN,System\LINE8\O8M1_TX,137156501,O8M1_TX,1,SIPLACE TX
129097,2022-11-09T12:25:05.730234Z,System\LINE_8,LINE_8,NaN,System\LINE8\O8M1_TX,137156501,O8M1_TX,1,SIPLACE TX
129118,2022-11-09T12:25:05.730234Z,System\LINE_8,LINE_8,NaN,System\LINE8\O8M1_TX,137156501,O8M1_TX,1,SIPLACE TX


# Which machine replace the unused machine.

In [14]:
raw_data.groupby('stationid').stationorder.value_counts()

stationid  stationorder
26         4               6009
78         2               6009
202        1               6009
946        4               6009
2560       2               6009
2651       2               6009
4885       1               1956
5899       1               6009
9918       2               6009
10887      2               6009
13942      3               6009
13970      3               6009
13974      1               1803
14704      1               2622
15378      2               6009
16054      1               6009
16056      1               6009
16067      1               6009
16122      1               6009
17714      2               1803
17748      3               6009
10000068   2               6009
10002710   3               6009
10002712   3               6009
137156501  1               1584
Name: count, dtype: int64

In [15]:
#Assuming the station oder is number of times the machine is replaced.
raw_data[(raw_data['stationorder'] > 1)]

,monitoringservicesendtime,linefullpath,linename,machineid,stationfullpath,stationid,stationname,stationorder,typename
1,2022-11-06T16:32:35.3242268Z,System\LINE_5,LINE_5,10000068.0,System\LINE5\O5M2_D4,10000068,O5M2_D4,2,SIPLACE D4
2,2022-11-06T16:32:35.3242268Z,System\LINE_5,LINE_5,13942.0,System\LINE5\O5M3_X4is,13942,O5M3_X4is,3,SIPLACE X4i S
3,2022-11-06T16:32:35.3242268Z,System\LINE_5,LINE_5,26.0,System\LINE5\O5M4_D1,26,O5M4_D1,4,SIPLACE D1
5,2022-11-06T16:32:35.3242268Z,System\LINE_8,LINE_8,9918.0,System\LINE8\O8M2_SX,9918,O8M2_SX,2,SIPLACE SX V2
6,2022-11-06T16:32:35.3242268Z,System\LINE_8,LINE_8,10002710.0,System\LINE8\O8M3_D1,10002710,O8M3_D1,3,SIPLACE D1
...,...,...,...,...,...,...,...,...,...
129940,2022-11-12T06:26:01.6368476Z,System\LINE_9,LINE_9,17748.0,System\LINE9\O9M3_SX,17748,O9M3_SX,3,SIPLACE SX V2
129942,2022-11-12T06:26:01.6368476Z,System\LINE_1,LINE_1,10887.0,System\LINE1\O1M2_X3s,10887,O1M2_X3s,2,SIPLACE X3 S
129944,2022-11-12T06:26:01.6368476Z,System\LINE_2,LINE_2,2560.0,System\LINE2\O2M2_D1,2560,O2M2_D1,2,SIPLACE D1
129945,2022-11-12T06:26:01.6368476Z,System\LINE_2,LINE_2,10002712.0,System\LINE2\O2M3_D1,10002712,O2M3_D1,3,SIPLACE D1
